
# Splitting big images into smaller pieces that are suitable to be printed out on A4 paper.

In [ ]:
from PIL import Image
import os

Image.MAX_IMAGE_PIXELS = None

In [ ]:
def split_img_A4(input_file_path, output_file_prefix):
    # A4 size = 210mm x 297mm
    a4_width_in = 8.27
    a4_height_in = 11.69

    # Open the image
    img = Image.open(input_file_path)
    img_width, img_height = img.size
    print(f"原始像素：{img_width} x {img_height}")
    origin_dpi = img.info.get('dpi')
    print(f"原始 dpi：{origin_dpi}")

    dpi = 300
    print(f"假设打印 dpi：{dpi}")
    # Calculate dimensions based on DPI
    a4_width_px = int(a4_width_in * dpi )
    a4_height_px = int(a4_height_in * dpi )

    pages1,w1,h1 = estimate_splits(img_width, img_height, a4_width_px, a4_height_px)
    pages2,w2,h2 = estimate_splits(img_width, img_height, a4_height_px, a4_width_px)

    print()
    print("由于", end='')
    if pages1 < pages2:
        print("方案 1 比较节约纸张", end='')
        a4_width_px = w1
        a4_height_px = h1
    elif pages1 == pages2:
        if max(w1,h1) < max(w2,h2):
            print("方案 1 在使用相同纸张情况下压缩较小", end='')
            a4_width_px = w1
            a4_height_px = h1
        else:
            print("方案 2 在使用相同纸张情况下压缩较小", end='')
            a4_width_px = w2
            a4_height_px = h2
    else:
        print("方案 2 比较节约纸张", end='')
        a4_width_px = w2
        a4_height_px = h2
    
    # Calculate number of tiles needed
    num_tiles_w = (img_width + a4_width_px - 1) // a4_width_px
    num_tiles_h = (img_height + a4_height_px - 1) // a4_height_px
    print(f"，总共切割成：{num_tiles_w} x {num_tiles_h} = {num_tiles_w*num_tiles_h} 张图片。每张大约：{w2} x {h2} 像素")
    
    
    # Split and save tiles
    for w in range(num_tiles_w):
        for h in range(num_tiles_h):
            # Calculate the coordinates for each tile
            left = w * a4_width_px
            top = h * a4_height_px
            right = left + a4_width_px
            bottom = top + a4_height_px
            
            # Crop the image to the current tile
            tile = img.crop((left, top, right, bottom))
            
            # Save the tile
            output_path = f"{output_file_prefix}_{h+1}-{w+1}.png"
            tile.save(output_path)
            print( '=' if a4_width_px>a4_height_px else '║', end='', flush=True)
        print()
    print("完成\n")

def estimate_splits(img_w,img_h,paper_w,paper_h):
    print()
    print(f"假如切割成：{paper_w} x {paper_h} ({'=' if paper_w>paper_h else '║'})")
    pages_w = (img_w + paper_w - 1) // paper_w
    width_res = (img_w % paper_w)/img_w
    print(f"水平页数：{pages_w} 最右一帧占总宽：{width_res:%}")
    pages_h = (img_h + paper_h - 1) // paper_h
    height_res = (img_h % paper_h)/img_h
    print(f"垂直页数：{pages_h} 最下一帧占总高：{height_res:%}")

    # 如果最后一帧内容小于tolerance，那就压缩画面节约纸张
    tolerance = 0.25
    if pages_h <=1 or pages_h <=1:
        return pages_w*pages_h, paper_w, paper_h
    if width_res <= tolerance and height_res <= tolerance:
        # 选取节约纸多的方式
        if pages_h > pages_w:
            #上下分页多就压宽度
            pages_h -= 1
            comparess_rate = width_res
        else:
            #左右分页多就压高度
            pages_w -= 1
            comparess_rate = height_res
    elif width_res <= tolerance:
        pages_h -= 1
        comparess_rate = width_res
    elif height_res <= tolerance:
        pages_w -= 1
        comparess_rate = height_res
    else:
        comparess_rate = 0
    
    if comparess_rate > 0:
        print(f"可以考虑压缩 {comparess_rate:.1%} 以提高纸张利用率")
        comparess_rate += 0.01
        paper_w *= (comparess_rate+1)
        paper_h *= (comparess_rate+1)

    return pages_w*pages_h, int(paper_w), int(paper_h)


split_img_A4("3.jpg", './tst')

原始像素：19821 x 31417
原始 dpi：(300, 300)
假设打印 dpi：300

假如切割成：2481 x 3507 (║)
水平页数：8 最右一帧占总宽：12.380808%
垂直页数：9 最下一帧占总高：10.698030%
可以考虑压缩 12.4% 以提高纸张利用率

假如切割成：3507 x 2481 (=)
水平页数：6 最右一帧占总宽：11.533222%
垂直页数：13 最下一帧占总高：5.236019%
可以考虑压缩 11.5% 以提高纸张利用率

由于方案 1 比较节约纸张，总共切割成：8 x 8 = 64 张图片。每张大约：3946 x 2791 像素
║║║║║║║║
║║║║║║║║
║║║║║║║║
║║║║║║║║
║║║║║║║║
║║║║║║║║
║║║║║║║║
║║║║║║║║
完成



In [ ]:
img_folder =    "/Users/I070019/Downloads/temp/道法思维导图（全）"
output_folder = "/Users/I070019/Downloads/temp/toBePrinted"

image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

os.makedirs(output_folder, exist_ok=True)

def find_image_files(directory):
    # 遍历当前目录中的所有项目
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # 检查是否是文件
        if os.path.isfile(file_path):
            # 获取扩展名并转换为小写
            ext = os.path.splitext(filename)[1].lower()
            
            # 如果是图像文件
            if ext in image_extensions:
                print(f"处理图像文件: {file_path}")
                name_from_path  = file_path[ len(img_folder)+1: -len(ext)].replace(os.path.sep, '_')
                split_img_A4(file_path, f"{output_folder}/{name_from_path}")
        
        # 检查是否是目录，如果是，则递归处理子目录
        elif os.path.isdir(file_path):
            find_image_files(file_path)

# 调用函数开始查找
find_image_files(img_folder)


处理图像文件: /Users/I070019/Downloads/temp/道法思维导图（全）/七年级道法思维导图/七下道法思维导图/第四单元 走进法治天地 .png
原始像素：3932 x 6080
原始 dpi：None
假设打印 dpi：300

假如切割成：2481 x 3507 (║)
水平页数：2 最右一帧占总宽：36.902340%
垂直页数：2 最下一帧占总高：42.319079%

假如切割成：3507 x 2481 (=)
水平页数：2 最右一帧占总宽：10.808749%
垂直页数：3 最下一帧占总高：18.388158%
可以考虑压缩 10.8% 以提高纸张利用率

由于方案 1 在使用相同纸张情况下压缩较小，总共切割成：2 x 2 = 4 张图片。每张大约：3921 x 2773 像素
║║
║║
完成

处理图像文件: /Users/I070019/Downloads/temp/道法思维导图（全）/七年级道法思维导图/七下道法思维导图/七下第三单元思维导图（新）.png
原始像素：3213 x 7160
原始 dpi：None
假设打印 dpi：300

假如切割成：2481 x 3507 (║)
水平页数：2 最右一帧占总宽：22.782446%
垂直页数：3 最下一帧占总高：2.039106%
可以考虑压缩 22.8% 以提高纸张利用率

假如切割成：3507 x 2481 (=)
水平页数：1 最右一帧占总宽：100.000000%
垂直页数：3 最下一帧占总高：30.698324%

由于方案 2 比较节约纸张，总共切割成：1 x 3 = 3 张图片。每张大约：3507 x 2481 像素
===
完成

处理图像文件: /Users/I070019/Downloads/temp/道法思维导图（全）/七年级道法思维导图/七下道法思维导图/第一单元 青春时光.png
原始像素：2833 x 8655
原始 dpi：None
假设打印 dpi：300

假如切割成：2481 x 3507 (║)
水平页数：2 最右一帧占总宽：12.424991%
垂直页数：3 最下一帧占总高：18.960139%
可以考虑压缩 12.4% 以提高纸张利用率

假如切割成：3507 x 2481 (=)
水平页数：1 最右一帧占总宽：100.00000